In [5]:
from utils import *
from imp import reload
import utils; reload(utils)
import vgg16; reload(vgg16)
from vgg16 import Vgg16
import keras.backend as K
%matplotlib inline

In [6]:
#Set path to sample/ path if desired
test_path = 'testing/' #We use all the test data
train_path='train/'
valid_path='valid/'

In [17]:
#import Vgg16 helper class
# vgg = Vgg16()

from keras.models import Model
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D

img_input = Input(shape=(224, 224, 3))

# Block 1
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

# Block 2
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

# Block 3
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

# Block 4
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

# Block 5
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

# Classification block
x = Flatten(name='flatten')(x)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(1000, activation='softmax', name='predictions')(x)

model = Model(img_input, x, name='my_vgg16')

In [19]:
from keras.utils.data_utils import get_file

file_name = 'vgg16_weights_tf_dim_ordering_tf_kernels.h5'
file_path = 'http://files.fast.ai/models/'
model.load_weights(get_file(file_name, file_path+file_name, cache_subdir='models'))

In [20]:
for layer in model.layers:
    layer.trainable = False
    
#Add a layer where input is the output of the  second last layer 
new_x = Dense(2, activation='softmax', name='predictions')(model.layers[-2].output)

#Then create the corresponding model 
my_model = Model(model.input, new_x)
my_model.summary()

my_model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [21]:
#Set constants. You can experiment with no_of_epochs to improve the model
batch_size=32
no_of_epochs=1

In [22]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_data = datagen.flow_from_directory(train_path, target_size=(224,224),
                class_mode='categorical', shuffle=True, batch_size=batch_size)

valid_data = ImageDataGenerator().flow_from_directory(valid_path, target_size=(224,224),
                class_mode='categorical', shuffle=True, batch_size=batch_size)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
import keras
from keras import backend as K
# K.clear_session()
    
for epoch in range(no_of_epochs):
    print("Running epoch:", epoch)
    steps_per_epoch = int(train_data.samples/train_data.batch_size)

    if steps_per_epoch == 0:
        steps_per_epoch = 1

    validation_steps = int(valid_data.samples/valid_data.batch_size)

    if validation_steps == 0:
        validation_steps = 1

    my_model.fit_generator(train_data, 
                             steps_per_epoch=steps_per_epoch,
                             epochs=1,
                             validation_data=valid_data, 
                             validation_steps=validation_steps)
    

my_model.save_weights('ft1.h5')     
print("Completed", no_of_epochs, "fit operations")

Running epoch: 0
Epoch 1/1
718/718 [==============================] - 262s - loss: 0.2113 - acc: 0.9310 - val_loss: 0.1826 - val_acc: 0.9516
